In [ ]:
def signals(Noversampl, depth0, oof, ofcont, FILT, Order, FiltType, FILTFACT, AutoPhase, ph0, IQ, noise):

    sstep = nptslambda/Noversampl
    depth = depth0 * 1e-6
    ph = int(nptslambda * ph0) #ph0 between 0 - 1
    
    #FRINGE
    er = np.sqrt(amp) #reference field
    es = R*np.sqrt(amp)*np.exp(1j*(2*np.pi*oof*(2*depth/c))) #sample field
    
    #I/Q + Balanced signal components
    sfI_1 = er + es
    sfQ_1 = er + es*np.exp(1j*np.pi/2)
    
    sfI_2 = er + es*np.exp(1j*np.pi)
    sfQ_2 = er + es*np.exp(1j*np.pi/2)*np.exp(1j*np.pi)
    
    #Noise
    sfI_1 = sfI_1 + np.sqrt(np.sqrt(amp))*np.random.randint(-10000,10000,len(oof))/10000*noise
    sfI_2 = sfI_2 + np.sqrt(np.sqrt(amp))*np.random.randint(-10000,10000,len(oof))/10000*noise

    sfQ_1 = sfQ_1 + np.sqrt(np.sqrt(amp))*np.random.randint(-10000,10000,len(oof))/10000*noise
    sfQ_2 = sfQ_2 + np.sqrt(np.sqrt(amp))*np.random.randint(-10000,10000,len(oof))/10000*noise
    
    #Balanced detection
    if BG:
        sfI = np.square(np.abs(sfI_1)) - np.square(np.abs(sfI_2))
        sfQ = np.square(np.abs(sfQ_1)) - np.square(np.abs(sfQ_2))
    else:
        sfI = np.square(np.abs(sfI_1))
        sfQ = np.square(np.abs(sfQ_1))
        
    if IQ:
        sf = sfI + 1j*sfQ
    else:
        sf = sfI
               
     #ANALOG FILTER
    if FILT:
        if FiltType == 1:
            b,a = signal.butter(Order, FILTFACT/nptslambda, 'low') 
        else:
            b,a = signal.cheby1(Order,0.5,FILTFACT/nptslambda,'low')

        sfilt = signal.filtfilt(b,a,sf);
        
    else:
        sfilt = sf
        
    #SAMPLING
    #cropping the middle one of three sweep
    ind = int(len(amp)/3)
    amp0 = amp[ind+1:ind+1+npts]; 

    oof = oof[ind+1:ind+1+npts];
    ofcont = ofcont[ind+1:ind+1+npts];
   
    inds = int(ind+1)
    inde = int(ind+1+npts)
    indstep = int(sstep)
    ss = sf[inds:inde]
       
    #FFT
    Ss0 = np.fft.fftshift(np.fft.ifft(ss, nfft)/Noversampl)
    Ssout = Ss0
    Ss = np.square(np.abs(Ssout))
     
    return(Ss)